---
# Exercise 6: Looping through a `GFF`
---
The purpose of Exercise 4 was to get you to understand how to translate a single gene from a GFF. But more often  you will be looping through a GFF file translating many genes. In this exercise, you will be looping through multiple genes in a GFF file and extract the sequences. 

### Instructions:
 - Loop through the GFF file `Chlamydomonas.gff` and extract the DNA coding sequence for each of the 5 genes and store them in a dictionary called **`coding_sequences`**
 - The key for each item should be the gene ID from the attributes column (excluding the 'ID=' and any possible the line breaks)
 - The key should refer to a DNA coding sequence in the positive frame (*i.e.* if the CDS is on the negative strand you have to reverse complement it). The sequence can be stored as a string or SeqObject but not a `SeqRecord`.
 
**Hints:**
 - You'll have to use the `.split` method from Lab 2 to extract columns from the table.
 - There are many ways of doing this task.
 - This is the most involved question you've been asked to do. Write out all the steps that would be necessary and then convert them to Python one by one.  
 - How can you test if you have an intact ORF?

--- 
Solution:

Given that there are multiple levels to this question, let's try to sequentially work through each of the instructions above.

1. Loop through the GFF file `Chlamydomonas.gff` and extract the DNA coding sequence for each of the 5 genes and store them in a dictionary called **`coding_sequences`**

Given that we're looping through a tab-separated file, we can immediately tell we'll be _looping through each line of the file_ and _using the `line.split` operation_. 'For each line of the file' is a tip off that we should use `file.readlines`, not `file.read`.

First, let's make sure that `line.split` is indeed the way to go. We can use a for loop and `break` to just print out the first line of the file, like so:

In [1]:
gff = open('Chlamydomonas.gff').readlines() # tacked on .readlines(), not .read()

for line in gff: # looping through lines
    print(line)
    print(line.split())
    break # stops the loop after the first iteration

chromosome_1	phytozome8_0	gene	333572	340433	.	+	.	ID=Cre01.g002050

['chromosome_1', 'phytozome8_0', 'gene', '333572', '340433', '.', '+', '.', 'ID=Cre01.g002050']


This looks good. Let's look at the next step we have to take.

1. Loop through the GFF file `Chlamydomonas.gff` and _extract the DNA coding sequence for each of the 5 genes and store them in a dictionary_ called **`coding_sequences`**

We have to get the CDS for each of the 5 genes in the file, and store them in a dictionary. Two things to pick up here - one is that 'for each' implies we'll be looping through this. _Remember that you should never have to manually do tasks such as retrieving the same chunk of information from multiple lines._

The second is that we're storing CDSs for different genes in a dictionary. The second instruction clarifies what the keys will be:

- The key for each item should be the gene ID from the attributes column (excluding the 'ID=' and any possible the line breaks)

What might immediately spring to mind is the `.split` method once again, except this time for a string. Alternatively, we could use string indexing to chop away the first three characters of this string. Let's go with the second since it might be easier. First, to make sure we don't mess up the Python indexing, let's actually do a quick test.

In [2]:
test = 'ID=Cre01.g002050'

print(test[3:])

Cre01.g002050


That looks like the gene name! Remember that you can always make a new cell and do quick tests like the one above for virtually anything.

Now, let's get started with our dictionary. We want to:

    create dictionary
    
    for each line in the file:
        split the line
        get the gene name
        remove the 'ID=' from the gene name
        save the gene name as a key in our dictionary
        
Let's convert the pseudocode here to code.

In [3]:
coding_sequences = {} # creates empty dict

for line in gff:
    line_list = line.split()
    gene_name = line_list[-1] # we saw above that it's the last element of the line
    gene_name = gene_name[3:] # this removes 'ID='
    coding_sequences[gene_name] = '' # this will be a string - let's put an empty one for now
    
print(coding_sequences)

{'Cre01.g002050': '', 'Cre01.g002100': '', 'Cre01.g002150': '', 'Cre01.g002200': '', 'g51': ''}


Our dictionary is all ready to go. Notice how we saved `''` and not `' '` to it - careful not to introduce spaces where you don't want them! Let's make sure `''` is actually empty:

In [4]:
print(len(''), # should be 0
      len('' + 'test')) # should be 4

0 4


Looks good. What's next?

3. The key should refer to a DNA coding sequence in the positive frame (*i.e.* if the CDS is on the negative strand you have to reverse complement it). The sequence can be stored as a string or SeqObject but not a `SeqRecord`.

So this is tougher - we want to get the CDS of each gene, of which we know there are multiple snippets, looking at the file/example from exercise 5. Second, if the gene is on the reverse complement, we have to account for that as well.

First, let's load the sequence data into our workspace with our good friend SeqIO. Since this file only contains one record, we can use `SeqIO.read()`:

In [5]:
from Bio import SeqIO

chlamy_fasta = SeqIO.read(open('Chlamydomonas.chromosome_1.fasta'), 'fasta')

Let's peek at our sequence, just to get a sense of what we're working with.

In [6]:
print(len(chlamy_fasta.seq))

400000


In [7]:
chlamy_fasta.seq[0:30]

Seq('GGGAACCAGCTACTAGATGGTTCGATTAGT', SingleLetterAlphabet())

Looks good. Now that we have that loaded in, let's try to write pseudocode for our next task.

Once again, we're looping through the file to get coordinates - that makes it our outermost for loop. For each line, if we match a gene name in our dictionary, we want to grab to corresponding sequence from the fasta file, being mindful, of course, of Python's weird 0-indexing.

Although - bear in mind that there are countless approaches to this, and what I have below is far from the only one (it's not even the same as how I solved this the first time).

    for each line in the file:
        split the line
        get the gene name
        get the type (CDS, exon, intron, etc) from that line
        if this line represents a CDS:
            get the coordinates
            get the sequence at those coordinates by indexing our SeqRecord
            add on that sequence to the corresponding dictionary entry

Let's convert this into code. It would be helpful to print out the line again:

In [8]:
for line in gff:
    print(line.split())
    break

['chromosome_1', 'phytozome8_0', 'gene', '333572', '340433', '.', '+', '.', 'ID=Cre01.g002050']


In [9]:
for line in gff:
    line_list = line.split()
    gene_name = line_list[-1][3:] # grabs gene name and chops off 'ID='
    line_type = line_list[2] # using coords from above
    if line_type == 'CDS':
        region_start = int(line_list[3]) - 1 # subtracting 1 for Python indexing
        region_end = int(line_list[4]) # remember line indices are strings
        sequence_snippet = chlamy_fasta.seq[region_start:region_end] # indexing sequence
        coding_sequences[gene_name] = coding_sequences[gene_name] + sequence_snippet # adds on
        # this last line is similar to counter += 1, or counter = counter + 1

Let's look at our dictionary:

In [10]:
coding_sequences

{'Cre01.g002050': Seq('ATGTCTCGGCTACCGGTGCCGCTGCCTGCCGTGGCCGCAGCGCTCGGCGTGGCC...TAG', SingleLetterAlphabet()),
 'Cre01.g002100': Seq('ATGCGTAGAGGTAGAGCTGGAGCCGGAGCTGCGGGTGGCCAGCACAAGAGCGCC...TGA', SingleLetterAlphabet()),
 'Cre01.g002150': Seq('CTATCCCGACTCCATGGCCGCCTCACCACCCGCGGCCCCAACCCCTTGCACCAT...CAT', SingleLetterAlphabet()),
 'Cre01.g002200': Seq('ATGGCTGACGACTATGATGAGGCTGGCGGGATGGAGGAGGATATTGGCATGGGC...TGA', SingleLetterAlphabet()),
 'g51': Seq('TCACAAGCGCCGCAACTGCTGCAGGGGACTGAACGGCGAGGCCACGCCGCCGCC...CAT', SingleLetterAlphabet())}

Two things to note about this:

1. Those all look like sequences - hooray!
2. What do we expect a CDS to look like? Notice how three of them start with ATG!

However, we haven't accounted for the reverse complement yet. Even if we'd forgotten about that for a moment, it's evident from what we see above - two of our sequences don't start with ATG.

Now for our final task - reverse complementing the sequences on the negative strand. There's multiple ways we could do this as well, not to mention ways it could have been included in the for loop above. I personally like the idea of making a dictionary of our own that contains gene names and the strands they're on - we can easily do that by copy pasting and modifying the above code.

In [11]:
strands = {}

for line in gff:
    line_list = line.split()
    gene_name = line_list[-1][3:] # grabs gene name and chops off 'ID='
    strand = line_list[6] # using coords from above
    strands[gene_name] = strand # saves orientation to our new dictionary
    
print(strands)

{'Cre01.g002050': '+', 'Cre01.g002100': '+', 'Cre01.g002150': '-', 'Cre01.g002200': '+', 'g51': '-'}


Now that we have this information, we could just eyeball the output above and reverse complement the corresponding genes in our other dictionary. However, since the keys are the same, we could also exploit that in a for loop.

In [12]:
for gene in coding_sequences.keys():
    if strands[gene] == '-':
        coding_sequences[gene] = coding_sequences[gene].reverse_complement()
        
coding_sequences

{'Cre01.g002050': Seq('ATGTCTCGGCTACCGGTGCCGCTGCCTGCCGTGGCCGCAGCGCTCGGCGTGGCC...TAG', SingleLetterAlphabet()),
 'Cre01.g002100': Seq('ATGCGTAGAGGTAGAGCTGGAGCCGGAGCTGCGGGTGGCCAGCACAAGAGCGCC...TGA', SingleLetterAlphabet()),
 'Cre01.g002150': Seq('ATGGTCTCGTGCCAAAACAGCAAGACATTAACATCACCTCGCTCCGCCTTGAAA...TAG', SingleLetterAlphabet()),
 'Cre01.g002200': Seq('ATGGCTGACGACTATGATGAGGCTGGCGGGATGGAGGAGGATATTGGCATGGGC...TGA', SingleLetterAlphabet()),
 'g51': Seq('ATGTCCCCGCGGCGACATGCCCTTCCGCTCCTGCTGGCGGCCGTGGCGGGAGGG...TGA', SingleLetterAlphabet())}

There we go! All of our sequences start with ATG. Of course, we didn't need to go through the process of making a second dictionary - simply looking at the output from two cells above and manually doing it for those two sequences would have been fine too.

Lastly, a clever way to check with certainty that these are indeed CDSs:

In [13]:
for gene in coding_sequences.keys():
    translation = coding_sequences[gene].translate() # translate the CDS! (don't overwrite dict, however)
    start = translation[0]
    end = translation[-1]
    print(start, end)

M *
M *
M *
M *
M *


Notice how each sequence not only starts with a methionine; it also ends with a stop codon. Neat!

For clarity, here's the full solution below without any comments:

In [14]:
from Bio import SeqIO

gff = open('Chlamydomonas.gff').readlines()
chlamy_fasta = SeqIO.read(open('Chlamydomonas.chromosome_1.fasta'), 'fasta')
coding_sequences = {}

for line in gff:
    line_list = line.split()
    gene_name = line_list[-1]
    gene_name = gene_name[3:]
    coding_sequences[gene_name] = ''
    
for line in gff:
    line_list = line.split()
    gene_name = line_list[-1][3:]
    line_type = line_list[2]
    if line_type == 'CDS':
        region_start = int(line_list[3]) - 1
        region_end = int(line_list[4])
        sequence_snippet = chlamy_fasta.seq[region_start:region_end]
        coding_sequences[gene_name] = coding_sequences[gene_name] + sequence_snippet
        
strands = {}

for line in gff:
    line_list = line.split()
    gene_name = line_list[-1][3:]
    strand = line_list[6]
    strands[gene_name] = strand
    
for gene in coding_sequences.keys():
    if strands[gene] == '-':
        coding_sequences[gene] = coding_sequences[gene].reverse_complement()
        
coding_sequences

{'Cre01.g002050': Seq('ATGTCTCGGCTACCGGTGCCGCTGCCTGCCGTGGCCGCAGCGCTCGGCGTGGCC...TAG', SingleLetterAlphabet()),
 'Cre01.g002100': Seq('ATGCGTAGAGGTAGAGCTGGAGCCGGAGCTGCGGGTGGCCAGCACAAGAGCGCC...TGA', SingleLetterAlphabet()),
 'Cre01.g002150': Seq('ATGGTCTCGTGCCAAAACAGCAAGACATTAACATCACCTCGCTCCGCCTTGAAA...TAG', SingleLetterAlphabet()),
 'Cre01.g002200': Seq('ATGGCTGACGACTATGATGAGGCTGGCGGGATGGAGGAGGATATTGGCATGGGC...TGA', SingleLetterAlphabet()),
 'g51': Seq('ATGTCCCCGCGGCGACATGCCCTTCCGCTCCTGCTGGCGGCCGTGGCGGGAGGG...TGA', SingleLetterAlphabet())}

Here's my original solution - less verbose, but possibly also less easy to understand:

In [15]:
from Bio import SeqIO
gff = open('Chlamydomonas.gff').readlines()
coding_sequences = {}
chromosome_1 = SeqIO.read(open('Chlamydomonas.chromosome_1.fasta'), 'fasta')

for line in gff:
    if line.split()[2] == 'gene':
        key = line.split()[-1][3:]
        coding_sequences[key] = ''

for key in coding_sequences.keys():
    for line in gff:
        sp = line.split()
        gene_name = sp[-1][3:]
        if gene_name == key and sp[2] == 'CDS':
            if sp[6] == '+':
                start = int(sp[3]) - 1
                end = int(sp[4])
                current_CDS = chromosome_1.seq[start:end]
                coding_sequences[key] = coding_sequences[key] + current_CDS
            elif sp[6] == '-':
                start = int(sp[3]) - 1
                end = int(sp[4])
                current_CDS = chromosome_1.seq[start:end].reverse_complement()
                coding_sequences[key] = current_CDS + coding_sequences[key] # add in reverse order
                
coding_sequences

{'Cre01.g002050': Seq('ATGTCTCGGCTACCGGTGCCGCTGCCTGCCGTGGCCGCAGCGCTCGGCGTGGCC...TAG', SingleLetterAlphabet()),
 'Cre01.g002100': Seq('ATGCGTAGAGGTAGAGCTGGAGCCGGAGCTGCGGGTGGCCAGCACAAGAGCGCC...TGA', SingleLetterAlphabet()),
 'Cre01.g002150': Seq('ATGGTCTCGTGCCAAAACAGCAAGACATTAACATCACCTCGCTCCGCCTTGAAA...TAG', SingleLetterAlphabet()),
 'Cre01.g002200': Seq('ATGGCTGACGACTATGATGAGGCTGGCGGGATGGAGGAGGATATTGGCATGGGC...TGA', SingleLetterAlphabet()),
 'g51': Seq('ATGTCCCCGCGGCGACATGCCCTTCCGCTCCTGCTGGCGGCCGTGGCGGGAGGG...TGA', SingleLetterAlphabet())}